# Communication Pipeline for Arduino UNO R4 to LEGO Spike Prime

In [ ]:
import sys
from typing import cast, TypeVar

TMessage = TypeVar("TMessage", bound="BaseMessage")

import cobs
from messages import *
from crc import crc

import asyncio
from bleak import BleakClient, BleakScanner
from bleak.backends.characteristic import BleakGATTCharacteristic
from bleak.backends.device import BLEDevice
from bleak.backends.scanner import AdvertisementData

SEARCH_TIMEOUT = 25.0
DEVICE_NOTIFICATION_INTERVAL_MS = 5000
DEVICE_PROGRAM_SLOT = 0


In [ ]:
PROGRAM_TO_SEND = \
"""# This is just an example program that we are sending
import motor
from hub import port
import time

motor.run_for_degrees(port.B, 360, 50)
time.sleep(0.5)
""".encode('utf8')

In [ ]:
"""
The LEGO® SPIKE™ Prime Hub exposes a BLE GATT service that contains two characteristics:
- RX (for receiving data)
- TX (for transmitting data)

The dict below cotains the UUIDS for the service and characteristics.
"""
LEGO_SPIKE_UUIDs = [
    {'Service' : '0000FD02-0000-1000-8000-00805F9B34FB'},
    {'RX'      : '0000FD02-0001-1000-8000-00805F9B34FB'},
    {'TX'      : '0000FD02-0002-1000-8000-00805F9B34FB'}
]

### Handshake and negotiation

In [ ]:
async def send_message(message: BaseMessage) -> None:
    print(f"Sending: {message}")
    payload = message.serialize()
    frame = cobs.pack(payload)

    # use the max_packet_size from the info response if available
    # otherwise, assume the frame is small enough to send in one packet
    packet_size = info_response.max_packet_size if info_response else len(frame)

    # send the frame in packets of packet_size
    for i in range(0, len(frame), packet_size):
        packet = frame[i : i + packet_size]
        await client.write_gatt_char(rx_char, packet, response=False)